In [23]:
import pandas as pd
import numpy as np
import random
from faker import Faker
import json
import sqlite3

# Инициализация Faker
fake = Faker('ru_RU')
np.random.seed(42)

# Генерация данных (аналогично предыдущему ответу)
chat_forum_ids = [f"CHAT_{i:03d}" for i in range(1, 101)] + [f"FORUM_{i:03d}" for i in range(1, 101)]
chat_forum_names = [
    "IT_Советы", "Travel_Базар", "Python_Код", "ЗП_Консультации", 
    "Java_Советы", "Документы_Эксперт", "Удалёнка_Лайф", 
    "AI_Вопросы", "Налоги_Советы", "Виза_НЗ", 
    "SQL_Оптимизация", "Нейросети_Тренды", "Стоматология_Помощь", 
    "Игры_Техника", "Аренда_Квартир"
]

data = []
for _ in range(100):
    chat_id = random.choice(chat_forum_ids)
    chat_name = random.choice(chat_forum_names)
    
    date_obj = fake.date_time_this_decade()
    date_str = date_obj.strftime('%Y-%m-%d')
    
    text = ' '.join([fake.word() for _ in range(random.randint(10, 15))])
    
    parent_id = random.choice([None, random.randint(1, 100)])
    author_login = f"x{fake.random_number(digits=5)}_{fake.user_name()}y"
    
    reactions = []
    for _ in range(random.randint(1, 3)):
        user = f"u{fake.random_number(digits=4)}"
        emoji = random.choice(['👍', '🤔', '🚀', '💡', '❓'])
        reaction_date = fake.date_time_between(start_date=date_obj, end_date='now')
        reactions.append({
            "user_login": user,
            "emoji": emoji,
            "reaction_date": reaction_date.strftime('%Y-%m-%dT%H:%M')
        })
    
    data.append({
        "id": len(data)+1,
        "chat_forum_id": chat_id,
        "chat_forum_name": chat_name,
        "text": text,
        "parent_post_id": parent_id,
        "author_login": author_login,
        "date": date_str,
        "reactions": json.dumps(reactions)
    })

df = pd.DataFrame(data)
# Создаем соединение с базой данных
conn = sqlite3.connect('chat_database.db')

# Создаем таблицу
with conn:
    conn.execute('''
        CREATE TABLE IF NOT EXISTS messages (
            id INTEGER PRIMARY KEY,
            chat_forum_id TEXT,
            chat_forum_name TEXT,
            text TEXT,
            parent_post_id INTEGER,
            author_login TEXT,
            date TEXT,
            reactions TEXT
        )
    ''')

# Загружаем данные в таблицу
df.to_sql('messages', conn, if_exists='replace', index=False)



In [29]:
# Пример SQL-запроса через Python
def run_query(query):
    result = pd.read_sql_query(query, conn)
    return result

# Пример 1: Вывести сообщения из IT_Советы
it_messages = run_query("SELECT * FROM messages WHERE chat_forum_name = 'IT_Советы'")
print("IT-сообщения:")
display(it_messages.head())

# Пример 2: Количество сообщений за 2023 год
count = run_query("SELECT COUNT(*) FROM messages WHERE date BETWEEN '2023-01-01' AND '2023-12-31'")
print(f"Количество сообщений в 2023: {count.iloc[0, 0]}")





IT-сообщения:


,id,chat_forum_id,chat_forum_name,text,parent_post_id,author_login,date,reactions
0,7,CHAT_075,IT_Советы,функция налево премьера налоговый командир рас...,58.0,x35447_sharovlevy,2022-06-13,"[{""user_login"": ""u4465"", ""emoji"": ""\u2753"", ""r..."
1,12,FORUM_012,IT_Советы,выдержать понятный белье шлем висеть снимать п...,42.0,x64047_savva_01y,2021-02-19,"[{""user_login"": ""u2471"", ""emoji"": ""\ud83d\udca..."
2,18,CHAT_055,IT_Советы,ручей миф покинуть банк поймать ответить доста...,92.0,x19481_zgalkiny,2025-01-09,"[{""user_login"": ""u1084"", ""emoji"": ""\u2753"", ""r..."
3,31,FORUM_026,IT_Советы,ложиться изредка отдел табак какой покинуть св...,NaN,x55624_lora1992y,2024-03-24,"[{""user_login"": ""u6185"", ""emoji"": ""\ud83e\udd1..."
4,45,CHAT_019,IT_Советы,бетонный тюрьма пастух мягкий промолчать короб...,NaN,x73211_foti2001y,2020-03-06,"[{""user_login"": ""u9313"", ""emoji"": ""\ud83d\udc4..."


Количество сообщений в 2023: 27


In [26]:
# Пример 3: Сообщения с реакцией "💡"
smart_reactions = run_query("SELECT * FROM messages WHERE reactions LIKE '%\"emoji\": \"💡\"%'")
print("Сообщения с умными реакциями:")
print(smart_reactions[['id', 'text', 'reactions']])

Сообщения с умными реакциями:
Empty DataFrame
Columns: [id, text, reactions]
Index: []
